In [1]:
import datetime
import numpy as np
import pandas as pd

from tqdm import tqdm, trange

from airsim.collections import AirObject, AirEnv, RadarSystem, ControlPoint, Supervisor
from airsim.time import Time

In [2]:
cp_data_dir = 'CP_data'

In [3]:
t_min = 0
t_max = 1000 * 1000
dt = 10

supervisor = Supervisor(
    air_env=AirEnv(air_objects=[
        AirObject(track=lambda t: np.array([200000 - 0.2 * t, 0, 7000])),
        AirObject(track=lambda t: np.array([0, 200000 - 0.2 * t, 8000])),
        AirObject(track=lambda t: np.array([-200000 + 0.2 * t, 0, 9000])),
        AirObject(track=lambda t: np.array([0, -200000 + 0.2 * t, 10000])),
    ]),
    radar_systems=[
        RadarSystem(position=np.array([0, 0, 0]),
                    detection_radius=75000,
                    error=5,
                    detection_fault_probability=0.05,
                    detection_period=500,
                    detection_delay=np.random.randint(0, 50)*10),
        RadarSystem(position=np.array([100000, 0, 0]),
                    detection_radius=75000,
                    error=5,
                    detection_fault_probability=0.05,
                    detection_period=500,
                    detection_delay=np.random.randint(0, 50)*10),
        RadarSystem(position=np.array([0, 100000, 0]),
                    detection_radius=75000,
                    error=5,
                    detection_fault_probability=0.05,
                    detection_period=500,
                    detection_delay=np.random.randint(0, 50)*10),
        RadarSystem(position=np.array([-100000, 0, 0]),
                    detection_radius=75000,
                    error=5,
                    detection_fault_probability=0.05,
                    detection_period=500,
                    detection_delay=np.random.randint(0, 50)*10),
        RadarSystem(position=np.array([0, -100000, 0]),
                    detection_radius=75000,
                    error=5,
                    detection_fault_probability=0.05,
                    detection_period=500,
                    detection_delay=np.random.randint(0, 50)*10)
    ]
)

# supervisor.run(t_min, t_max, dt)

cp_data = supervisor.get_data()
timestamp = datetime.datetime.now().strftime('%d%m%H%M%S')
# cp_data.to_csv(f'{cp_data_dir}/train/data_{timestamp}.csv')

In [4]:
t_min = 0
t_max = 3000 * 1000
dt = 10

supervisor = Supervisor(
    air_env=AirEnv(air_objects=[
        AirObject(track=lambda t: np.array([-100000 + 0.2 * t, 20000 * np.sin((2*np.pi/600000)*t), 10000]))
    ]),
    radar_systems=[
        RadarSystem(position=np.array([0, 0, 0]),
                    detection_radius=75000,
                    error=5,
                    detection_fault_probability=0.05,
                    detection_period=500,
                    detection_delay=np.random.randint(0, 50)*10),
        RadarSystem(position=np.array([100000, 0, 0]),
                    detection_radius=75000,
                    error=5,
                    detection_fault_probability=0.05,
                    detection_period=500,
                    detection_delay=np.random.randint(0, 50)*10),
        RadarSystem(position=np.array([200000, 0, 0]),
                    detection_radius=75000,
                    error=5,
                    detection_fault_probability=0.05,
                    detection_period=500,
                    detection_delay=np.random.randint(0, 50)*10),
        RadarSystem(position=np.array([300000, 0, 0]),
                    detection_radius=75000,
                    error=5,
                    detection_fault_probability=0.05,
                    detection_period=500,
                    detection_delay=np.random.randint(0, 50)*10),
        RadarSystem(position=np.array([400000, 0, 0]),
                    detection_radius=75000,
                    error=5,
                    detection_fault_probability=0.05,
                    detection_period=500,
                    detection_delay=np.random.randint(0, 50)*10)
    ]
)

supervisor.run(t_min, t_max, dt)

cp_data = supervisor.get_data()
timestamp = datetime.datetime.now().strftime('%d%m%H%M%S')
# cp_data.to_csv(f'{cp_data_dir}/train/data_{timestamp}.csv')

Running system: 100%|█████████████████████████████████████| 300001/300001 [01:21<00:00, 3674.19it/s]


In [5]:
def transform_cp_data(data, error, velocity, acceleration):
    data = data.sort_values(by=['rs_id', 'id', 'time']).reset_index(drop=True)
    
    data = data[
        ['rs_id', 'id', 'time', 'x', 'y', 'z'] + \
        (['x_err', 'y_err', 'z_err'] if error else []) + \
        (['v_x_est', 'v_y_est', 'v_z_est'] if velocity else []) + \
        (['a_x_est', 'a_y_est', 'a_z_est'] if acceleration else [])
    ]

    data = data.dropna()

    return data

In [6]:
def get_track(data, rs_id, id, t_min=None, t_max=None, drop_ids=True):
    track = data[(data['rs_id'] == rs_id) & (data['id'] == id)]
    if t_min is not None:
        track = track[track['time'] >= t_min]
    if t_max is not None:
        track = track[track['time'] <= t_max]
    if drop_ids:
        track = track.drop(columns=['rs_id', 'id'])
    return track

In [7]:
def get_tracks_timeranges_intersection(track_1, track_2):
    t_min_1, t_max_1 = track_1['time'].min(), track_1['time'].max()
    t_min_2, t_max_2 = track_2['time'].min(), track_2['time'].max()

    t_min, t_max = max(t_min_1, t_min_2), min(t_max_1, t_max_2)
    return t_min, t_max

In [16]:
cp_data = transform_cp_data(cp_data, error=False, velocity=False, acceleration=False)

In [17]:
track_1 = get_track(cp_data, 0, 0)
track_2 = get_track(cp_data, 1, 0)

In [18]:
track_1

,time,x,y,z
0,142300,-71543.526093,19930.410457,10001.575853
1,142800,-71438.206579,19945.806078,10004.812586
2,143300,-71338.018069,19953.130471,9997.088409
3,143800,-71236.107571,19952.921486,10003.266814
4,144300,-71144.683206,19968.308327,9999.343768
...,...,...,...,...
1378,868300,73660.228571,6522.691569,9998.507915
1379,868800,73763.039884,6414.914880,9999.669339
1380,869300,73860.071738,6315.451108,9998.918404
1381,869800,73962.028708,6220.309478,10002.191322


In [19]:
track_2

,time,x,y,z
1383,630040,26003.124940,6192.550071,9999.731441
1384,630540,26110.249667,6286.003085,10003.724092
1385,631040,26209.342823,6391.362637,9998.492294
1386,631540,26308.890748,6487.940576,10003.302961
1387,632040,26411.567087,6581.620910,9998.027353
...,...,...,...,...
2758,1356040,171203.480078,19963.375709,10003.499325
2759,1356540,171309.414046,19957.938546,9995.228371
2760,1357040,171409.131434,19941.104862,9998.216908
2761,1357540,171511.333759,19937.939420,9997.345733


In [20]:
t_min, t_max = get_tracks_timeranges_intersection(track_1, track_2)

In [21]:
print(t_min, t_max)

630040 870300


In [22]:
track_parts_pairs = []
current_t_max = t_max
track_length = 32

In [23]:
while current_t_max > t_min:
    track_1 = track_1[track_1['time'] < current_t_max]
    track_2 = track_2[track_2['time'] < current_t_max]
    
    if len(track_1) >= track_length and len(track_2) >= track_length:
        track_1_part = track_1.tail(track_length)
        track_1 = track_1.drop(track_1.tail(track_length).index)
        
        track_2_part = track_2.tail(track_length)
        track_2 = track_2.drop(track_2.tail(track_length).index)
        
        current_t_max = min(track_1['time'].max(), track_2['time'].max())

        track_parts_pairs.append((track_1_part, track_2_part))
    else:
        break

In [24]:
track_parts_pairs

[(        time             x            y             z
  1350  853800  70762.970959  9306.965932  10004.510545
  1351  854300  70858.488330  9213.280241  10003.296030
  1352  854800  70958.962002  9114.395220   9996.557189
  1353  855300  71062.693487  9022.693260  10002.609750
  1354  855800  71160.389381  8933.065789   9999.182052
  1355  856300  71256.122050  8832.419708   9996.074412
  1356  856800  71358.748531  8744.338964   9999.667085
  1357  857300  71459.249324  8644.685176   9995.937758
  1358  857800  71562.723721  8555.091029  10001.412238
  1359  858300  71660.903910  8459.032053   9996.589103
  1360  858800  71761.898117  8361.876437  10003.514650
  1361  859300  71857.816249  8268.028337  10003.625939
  1362  859800  71956.546473  8175.070488   9998.568406
  1363  860300  72060.312595  8075.633246   9997.232383
  1364  860800  72155.327946  7978.927522  10001.451671
  1365  861300  72260.534206  7888.109408  10002.594645
  1366  861800  72357.161633  7790.098667   9996